In [12]:
import timm
import json
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset, random_split
from urllib.request import urlopen
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from torch.utils.data import Dataset
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import precision_score

In [11]:
import pandas as pd

binary_training = pd.read_csv("convnext_v2_outputs/Single_Model/Single_model_binary_repeat.csv", index_col = "Unnamed: 0")
binary_holdout = pd.read_csv("convnext_v2_outputs/Single_Model/Single_model_binary_holdout_repeat.csv", index_col = "Unnamed: 0")
multiclass_training = pd.read_csv("convnext_v2_outputs/Single_Model/Multiclass/Epochs_summary.csv", index_col = "Unnamed: 0")
multiclass_holdout = pd.read_csv("convnext_v2_outputs/Single_Model/Multiclass/Holdout_perclass.csv", index_col = "Unnamed: 0")

In [27]:
multiclass_holdout

,Class,Holdout_Recall,Holdout_Specificity,Holdout_F1,Holdout_AUC
0,adenosis,0.587156,0.998930,0.731429,0.993702
1,ductal_carcinoma,0.976219,0.808267,0.873404,0.966983
2,fibroadenoma,0.912727,0.940693,0.800638,0.976666
3,lobular_carcinoma,0.630303,0.974628,0.660317,0.972388
4,mucinous_carcinoma,0.567251,0.995019,0.700361,0.944741
5,papillary_carcinoma,0.456954,0.996716,0.610619,0.964517
6,phyllodes_tumor,0.370000,0.995208,0.506849,0.959084
7,tubular_adenoma,0.807229,0.994481,0.864516,0.993368


In [24]:
binary_training["Train_accuracy"] = binary_training["Train_Acc"]
binary_training["Test_accuracy"] = binary_training["Test_Acc"]
multiclass_training["Train_accuracy"] = multiclass_training["2"]
multiclass_training["Test_accuracy"] = multiclass_training["4"]
multiclass_training["Epoch"] = binary_training["Epoch"][0:multiclass_training.shape[0]]

KeyError: 'Train_Acc'

In [25]:
binary_training = binary_training[["Epoch", "Train_accuracy", "Test_accuracy"]]
multiclass_training = multiclass_training[["Epoch", "Train_accuracy", "Test_accuracy"]]
multiclass_training

,Epoch,Train_accuracy,Test_accuracy
0,Epoch1,0.517869,0.602158
1,Epoch2,0.670275,0.709373
2,Epoch3,0.751546,0.687121
3,Epoch4,0.815808,0.738368
4,Epoch5,0.841753,0.753877
5,Epoch6,0.888144,0.832097
6,Epoch7,0.908591,0.847606
7,Epoch8,0.938144,0.852326
8,Epoch9,0.943471,0.833446
9,Epoch10,0.945533,0.846932


In [29]:
binary_training.to_csv("../Final_Results/ConvNeXT-V2/binary_perepoch.csv")
multiclass_training.to_csv("../Final_Results/ConvNeXT-V2/multiclass_perepoch.csv")
binary_holdout.to_csv("../Final_Results/ConvNeXT-V2/binary_holdout.csv")
multiclass_holdout.to_csv("../Final_Results/ConvNeXT-V2/multiclass_holdout.csv")

In [13]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts to [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225]),
])

class BreakHisDataset(Dataset): # Subclass Dataset, which is required for using DataLoader
    def __init__(self, csv_path, transform=None):
        self.df = pd.read_csv(csv_path)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filepath']
        label = self.df.loc[idx, 'label']
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)
    
test_dataset = BreakHisDataset(csv_path="../data/new_holdout.csv", transform=transform) # Load the data

# Create iterable data loaders

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False) # shuffle=False for consistent evaluation

In [32]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd
import timm
import torch

model = timm.create_model('convnextv2_atto.fcmae', pretrained=False, num_classes=8) # model architecture
model.load_state_dict(torch.load("convnext_v2_outputs/Single_Model/Multiclass/best_model_multiclass.pth")) # saved weights
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts to [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225]),
])

class BreakHisDataset(Dataset): # Subclass Dataset, which is required for using DataLoader
    def __init__(self, csv_path, transform=None):
        self.df = pd.read_csv(csv_path)
        self.transform = transform
        self.le = LabelEncoder()
        self.df['label_diagnosis'] = self.le.fit_transform(self.df['tumor_subtype'])
        self.class_names = list(self.le.classes_)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filepath']
        label = self.df.loc[idx, 'label']
        subtype = self.df.loc[idx, 'label_diagnosis']   
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long), torch.tensor(subtype, dtype=torch.long)

test_dataset = BreakHisDataset(csv_path="../data/new_holdout.csv", transform=transform) # Load the data
holdout_loader = DataLoader(test_dataset, batch_size=16, shuffle=False) # shuffle=False for consistent evaluation

with torch.no_grad():
    for image, _, label in holdout_loader:
        outputs = model(image)
        all_labels.append(label)
        probs = torch.softmax(outputs, dim=1)
        all_probs.append(probs.detach())
        _, preds = torch.max(outputs, 1)
        all_preds.append(preds)

all_holdout_labels = torch.cat(all_labels).detach().numpy()
all_holdout_preds = torch.cat(all_preds).detach().numpy()

precision = precision_score(all_holdout_labels, all_holdout_preds, average=None)
precision

array([0.44043593, 0.73573017, 0.71306818, 0.69333333, 0.91509434,
       0.92      , 0.80434783, 0.93055556])

In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts to [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225]),
])

class BreakHisDataset(Dataset): # Subclass Dataset, which is required for using DataLoader
    def __init__(self, csv_path, transform=None):
        self.df = pd.read_csv(csv_path)
        self.transform = transform
        self.le = LabelEncoder()
        self.df['label_diagnosis'] = self.le.fit_transform(self.df['tumor_subtype'])
        self.class_names = list(self.le.classes_)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filepath']
        label = self.df.loc[idx, 'label']
        subtype = self.df.loc[idx, 'label_diagnosis']   
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long), torch.tensor(subtype, dtype=torch.long)

test_dataset = BreakHisDataset(csv_path="../data/new_holdout.csv", transform=transform) # Load the data
holdout_loader = DataLoader(test_dataset, batch_size=16, shuffle=False) # shuffle=False for consistent evaluation

model.eva
with torch.no_grad():
    for image, _, label in holdout_loader:
        outputs = model(image)
        all_labels.append(label)
        all_probs.append(probs.detach())
        _, preds = torch.max(outputs, 1)
        all_preds.append(preds)

all_holdout_labels = torch.cat(all_labels).detach().numpy()
all_holdout_preds = torch.cat(all_preds).detach().numpy()

from sklearn.metrics import precision_score

precision = precision_score(all_holdout_labels, all_holdout_preds, average=None)
precision

array([0.42697495, 0.72992925, 0.71306818, 0.69333333, 0.91509434,
       0.92      , 0.80434783, 0.93055556])

In [28]:
precision

array([0.39788875, 0.71361564, 0.71306818, 0.69333333, 0.91509434,
       0.92      , 0.80434783, 0.93055556])

In [29]:
test_dataset.class_names

['adenosis',
 'ductal_carcinoma',
 'fibroadenoma',
 'lobular_carcinoma',
 'mucinous_carcinoma',
 'papillary_carcinoma',
 'phyllodes_tumor',
 'tubular_adenoma']

In [ ]:
cm_holdout = confusion_matrix(all_holdout_labels, all_holdout_preds)